<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Meta/blob/master/tutorials/1-Introduction/China_A_share_market_tushare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Quantitative trading in China A stock market with FinRL

Install FinRL

In [ ]:
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git


Install other libraries

In [ ]:
!pip install stockstats
!pip install tushare
#install talib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz 
!tar xvzf ta-lib-0.4.0-src.tar.gz
import os
os.chdir('ta-lib') 
!./configure --prefix=/usr
!make
!make install
#!sudo make install # Sometimes it need root 
os.chdir('../')
!pip install TA-Lib

In [20]:
%cd /
!git clone https://github.com/AI4Finance-Foundation/FinRL-Meta
%cd /FinRL-Meta/

/
fatal: destination path 'FinRL-Meta' already exists and is not an empty directory.
/FinRL-Meta


##Import Modules

In [21]:
import warnings

warnings.filterwarnings("ignore")

import pandas as pd 
from IPython import display

display.set_matplotlib_formats("svg")

from meta import config 
from meta.data_processor import DataProcessor 
from main import check_and_make_directories 
from meta.data_processors.tushare import Tushare, ReturnPlotter 
from meta.env_stock_trading.env_stocktrading_China_A_shares import StockTradingEnv 
from agents.stablebaselines3_models import DRLAgent 
import os 
from typing import List 
from argparse import ArgumentParser 
from meta import config 
from meta.config_tickers import DOW_30_TICKER 
from meta.config import ( DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR, INDICATORS, TRAIN_START_DATE, TRAIN_END_DATE, TEST_START_DATE, TEST_END_DATE, TRADE_START_DATE, TRADE_END_DATE, ERL_PARAMS, RLlib_PARAMS, SAC_PARAMS, ALPACA_API_KEY, ALPACA_API_SECRET, ALPACA_API_BASE_URL, )

import pyfolio
from pyfolio import timeseries

pd.options.display.max_columns = None

print("ALL Modules have been imported!")

ALL Modules have been imported!


##Create Folders

In [22]:
import os

''' 
use check_and_make_directories() to replace the following

if not os.path.exists("./datasets"): 
  os.makedirs("./datasets") 
if not os.path.exists("./trained_models"): 
  os.makedirs("./trained_models") 
if not os.path.exists("./tensorboard_log"): 
  os.makedirs("./tensorboard_log") 
if not os.path.exists("./results"): 
  os.makedirs("./results") 
'''

check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

##Download data, cleaning and feature engineering

In [23]:
ticker_list = ['600000.SH', '600009.SH', '600016.SH', '600028.SH', '600030.SH', '600031.SH', '600036.SH', '600050.SH', '600104.SH', '600196.SH', '600276.SH', '600309.SH', '600519.SH', '600547.SH', '600570.SH']

TRAIN_START_DATE = '2015-01-01' 
TRAIN_END_DATE= '2019-08-01' 
TRADE_START_DATE = '2019-08-01' 
TRADE_END_DATE = '2020-01-03'

TIME_INTERVAL = "1d" 
kwargs = {} 
kwargs['token'] = '27080ec403c0218f96f388bca1b1d85329d563c91a43672239619ef5' 
p = DataProcessor(data_source='tushare', start_date=TRAIN_START_DATE, end_date=TRADE_END_DATE, time_interval=TIME_INTERVAL, **kwargs)

tushare successfully connected


###Download and Clean

In [24]:
p.download_data(ticker_list=ticker_list)

p.clean_data()

100%|██████████| 15/15 [00:17<00:00,  1.18s/it]


Shape of DataFrame:  (17960, 8)
Shape of DataFrame:  (18315, 8)


###Add technical indicator

In [ ]:
p.add_technical_indicator(config.INDICATORS) 
p.clean_data()

#print(f"p.dataframe: {p.dataframe}")

##Split training dataset

In [26]:
train = p.data_split(p.dataframe, TRAIN_START_DATE, TRAIN_END_DATE) 

print(f"len(train.tic.unique()): {len(train.tic.unique())}")

len(train.tic.unique()): 15


In [27]:
print(f"train.tic.unique(): {train.tic.unique()}")

train.tic.unique(): ['600000.SH' '600009.SH' '600016.SH' '600028.SH' '600030.SH' '600031.SH'
 '600036.SH' '600050.SH' '600104.SH' '600196.SH' '600276.SH' '600309.SH'
 '600519.SH' '600547.SH' '600570.SH']


In [28]:
print(f"train.head(): {train.head()}")

train.head():          tic        date  index   open   high    low  close      volume  day  \
0  600000.SH  2015-01-08     45  15.87  15.88  15.20  15.25  3306271.72  3.0   
0  600009.SH  2015-01-08     46  20.18  20.18  19.73  20.00   198117.45  3.0   
0  600016.SH  2015-01-08     47  10.61  10.66  10.09  10.20  4851684.17  3.0   
0  600028.SH  2015-01-08     48   7.09   7.41   6.83   6.85  8190902.35  3.0   
0  600030.SH  2015-01-08     49  36.40  36.70  34.68  35.25  6376268.69  3.0   

       macd    boll_ub    boll_lb     rsi_30      cci_30       dx_30  \
0 -0.032571  16.617911  15.012089   6.058641 -125.593009   23.014040   
0 -0.016008  20.663897  19.736103  12.828915  -90.842491  100.000000   
0 -0.018247  10.957604   9.997396  11.862558  -99.887006  100.000000   
0 -0.008227   7.342000   6.743000  27.409248   36.578171   64.934862   
0  0.032910  36.576444  33.808556  61.517448   47.947020  100.000000   

   close_30_sma  close_60_sma  
0       15.8150       15.8150  
0       

In [29]:
print(f"train.shape: {train.shape}")

train.shape: (16695, 17)


In [30]:
stock_dimension = len(train.tic.unique()) 
state_space = stock_dimension * (len(config.INDICATORS) + 2) + 1 

print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 15, State Space: 151


##Train

In [31]:
env_kwargs = { "stock_dim": stock_dimension, "hmax": 1000, "initial_amount": 1000000, "buy_cost_pct": 6.87e-5, "sell_cost_pct": 1.0687e-3, "reward_scaling": 1e-4, "state_space": state_space, "action_space": stock_dimension, "tech_indicator_list": config.INDICATORS, "print_verbosity": 1, "initial_buy": True, "hundred_each_trade": True }

e_train_gym = StockTradingEnv(df=train, **env_kwargs)

In [ ]:
env_train, _ = e_train_gym.get_sb_env() 

print(f"print(type(env_train)): {print(type(env_train))}")

###DDPG

In [33]:
agent = DRLAgent(env=env_train) 
DDPG_PARAMS = { "batch_size": 256, "buffer_size": 50000, "learning_rate": 0.0005, "action_noise": "normal", } 
POLICY_KWARGS = dict(net_arch=dict(pi=[64, 64], qf=[400, 300])) 
model_ddpg = agent.get_model("ddpg", model_kwargs=DDPG_PARAMS, policy_kwargs=POLICY_KWARGS)

trained_ddpg = agent.train_model(model=model_ddpg, tb_log_name='ddpg', total_timesteps=10000)

{'batch_size': 256, 'buffer_size': 50000, 'learning_rate': 0.0005, 'action_noise': NormalActionNoise(mu=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], sigma=[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1])}
Using cpu device
Logging to tensorboard_log/ddpg/ddpg_1
Episode: 2
day: 1112, episode: 2
begin_total_asset: 1000000.00
end_total_asset: 2109049.67
total_reward: 1109049.67
total_cost: 12151.48
total_trades: 16679
Sharpe: 0.726
Episode: 3
day: 1112, episode: 3
begin_total_asset: 1000000.00
end_total_asset: 1873632.31
total_reward: 873632.31
total_cost: 620.69
total_trades: 16680
Sharpe: 0.650
Episode: 4
day: 1112, episode: 4
begin_total_asset: 1000000.00
end_total_asset: 1480411.95
total_reward: 480411.95
total_cost: 488.05
total_trades: 16680
Sharpe: 0.471
Episode: 5
day: 1112, episode: 5
begin_total_asset: 1000000.00
end_total_asset: 1473792.94
total_reward: 473792.94
total_cost: 488.06
total_trades: 16680
Sharpe: 0.467
-----------------------------------
| time/    

###A2C

In [34]:
agent = DRLAgent(env=env_train) 
model_a2c = agent.get_model("a2c")

trained_a2c = agent.train_model(model=model_a2c, tb_log_name='a2c', total_timesteps=50000)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Logging to tensorboard_log/a2c/a2c_1
--------------------------------------
| time/                 |            |
|    fps                | 251        |
|    iterations         | 100        |
|    time_elapsed       | 1          |
|    total_timesteps    | 500        |
| train/                |            |
|    entropy_loss       | -21.3      |
|    explained_variance | -0.0322    |
|    learning_rate      | 0.0007     |
|    n_updates          | 99         |
|    policy_loss        | -2.66      |
|    reward             | -0.5146969 |
|    std                | 1          |
|    value_loss         | 2.24       |
--------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 248       |
|    iterations         | 200       |
|    time_elapsed       | 4         |
|    total_timesteps    | 1000      |
| train/                |           

##Trade

In [35]:
trade = p.data_split(p.dataframe, TRADE_START_DATE, TRADE_END_DATE) 
env_kwargs = { "stock_dim": stock_dimension, "hmax": 1000, "initial_amount": 1000000, "buy_cost_pct": 6.87e-5, "sell_cost_pct": 1.0687e-3, "reward_scaling": 1e-4, "state_space": state_space, "action_space": stock_dimension, "tech_indicator_list": config.INDICATORS, "print_verbosity": 1, "initial_buy": False, "hundred_each_trade": True } 
e_trade_gym = StockTradingEnv(df=trade, **env_kwargs)

In [36]:
df_account_value, df_actions = DRLAgent.DRL_prediction(model=trained_ddpg, environment=e_trade_gym)

Episode: 2
day: 103, episode: 2
begin_total_asset: 1000000.00
end_total_asset: 952511.32
total_reward: -47488.68
total_cost: 68.68
total_trades: 608
Sharpe: -0.366
hit end!


In [37]:
df_actions.to_csv("action.csv", index=False) 
print(f"df_actions: {df_actions}")

df_actions:             600000.SH  600009.SH  600016.SH  600028.SH  600030.SH  600031.SH  \
date                                                                           
2019-08-01          0          0       1000       1000          0       1000   
2019-08-02          0          0       1000       1000          0       1000   
2019-08-05          0          0       1000       1000          0       1000   
2019-08-06          0          0       1000       1000          0       1000   
2019-08-07          0          0       1000       1000          0       1000   
...               ...        ...        ...        ...        ...        ...   
2019-12-25          0          0          0          0          0          0   
2019-12-26          0          0          0          0          0          0   
2019-12-27          0          0          0          0          0          0   
2019-12-30          0          0          0          0          0          0   
2019-12-31          0       

##Backtest

###matplotlib inline

In [38]:
plotter = ReturnPlotter(df_account_value, trade, TRADE_START_DATE, TRADE_END_DATE)
plotter.plot()

In [ ]:
# ticket: SSE 50：000016
plotter.plot("000016")

###CSI 300

In [ ]:
baseline_df = plotter.get_baseline("399300")

In [ ]:
daily_return = plotter.get_return(df_account_value)
daily_return_base = plotter.get_return(baseline_df, value_col_name="close")

perf_func = timeseries.perf_stats 
perf_stats_all = perf_func(returns=daily_return, factor_returns=daily_return_base, positions=None, transactions=None, turnover_denom="AGB")
print("==============DRL Strategy Stats===========")
print(f"perf_stats_all: {perf_stats_all}")

In [ ]:
daily_return = plotter.get_return(df_account_value)
daily_return_base = plotter.get_return(baseline_df, value_col_name="close")

perf_func = timeseries.perf_stats
perf_stats_all = perf_func(returns=daily_return_base, factor_returns=daily_return_base, positions=None, transactions=None, turnover_denom="AGB")

print("==============Baseline Strategy Stats===========")

print(f"perf_stats_all: {perf_stats_all}")